In [1]:
# Experiment No.  3
# Write a program to demonstrate the working of the decision tree based on ID3 algorithm. Use an appropriate data set for building the decision tree and apply this knowledge to classify a new sample

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from pandas import read_csv

In [4]:
import pandas as pd
import numpy as np
df_tennis = pd.read_csv('Rounak_Exp3.csv')

In [5]:
df_tennis

,outlook,temp,humidity,windy,PlayTennis
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [6]:
type(df_tennis)

pandas.core.frame.DataFrame

In [7]:
df_tennis.dtypes

outlook       object
temp          object
humidity      object
windy           bool
PlayTennis    object
dtype: object

In [8]:
import csv
a = []
print("\n The Given Training Data Set")
with open('Rounak_Exp3.csv', 'r') as csvFile:
    reader = csv.reader(csvFile)
    for row in reader:
        a.append (row)
        print(row)
num_attributes = len(a[0])-1


 The Given Training Data Set
['outlook', 'temp', 'humidity', 'windy', 'PlayTennis']
['sunny', 'hot', 'high', 'FALSE', 'no']
['sunny', 'hot', 'high', 'TRUE', 'no']
['overcast', 'hot', 'high', 'FALSE', 'yes']
['rainy', 'mild', 'high', 'FALSE', 'yes']
['rainy', 'cool', 'normal', 'FALSE', 'yes']
['rainy', 'cool', 'normal', 'TRUE', 'no']
['overcast', 'cool', 'normal', 'TRUE', 'yes']
['sunny', 'mild', 'high', 'FALSE', 'no']
['sunny', 'cool', 'normal', 'FALSE', 'yes']
['rainy', 'mild', 'normal', 'FALSE', 'yes']
['sunny', 'mild', 'normal', 'TRUE', 'yes']
['overcast', 'mild', 'high', 'TRUE', 'yes']
['overcast', 'hot', 'normal', 'FALSE', 'yes']
['rainy', 'mild', 'high', 'TRUE', 'no']


In [9]:
from collections import Counter
def entropy_list(a_list):
    cnt = Counter(x for x in a_list)
    num_instance = len(a_list)*1.0
    probs = [x/num_instance for x in cnt.values()]
    return entropy(probs)

In [10]:
import math
def entropy(probs):
    return sum([-prob*math.log(prob,2) for prob in probs])

In [11]:
def info_gain(df,split,target,trace=0):
    df_split = df.groupby(split)
    nobs = len(df.index)*1.0
    df_agg_ent = df_split.agg({ target:[entropy_list, lambda x: len(x)/nobs] })
#     print(df_agg_ent)
    df_agg_ent.columns = ['Entropy','PropObserved']
    new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent["PropObserved"])
    old_entropy = entropy_list(df[target])
    return old_entropy - new_entropy

In [12]:
def id3(df,target,attribute_name,default_class = None):
    cnt = Counter(x for x in df[target])
    if len(cnt)==1:
        return next(iter(cnt))
    elif df.empty or (not attribute_name):
        return default_class
    else:
        default_class = max(cnt.keys())
        gains = [info_gain(df,attr,target) for attr in attribute_name]
        index_max = gains.index(max(gains))
        best_attr = attribute_name[index_max]
        tree = { best_attr:{ } }
        remaining_attr = [x for x in attribute_name if x!=best_attr]
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,target,remaining_attr,default_class)
            tree[best_attr][attr_val] = subtree
        return tree

In [13]:
def classify(instance,tree,default = None):
    attribute = next(iter(tree))
    if instance[attribute] in tree[attribute].keys():
        result = tree[attribute][instance[attribute]]
        if isinstance(result,dict):
            return classify(instance,result)
        else:
            return result
    else:
        return default

In [14]:
attribute_names = list(df_tennis.columns)
attribute_names.remove('PlayTennis') #Remove the class attribute
tree = id3(df_tennis,'PlayTennis',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")
print(tree)



The Resultant Decision Tree is :

{'outlook': {'overcast': 'yes', 'rainy': {'windy': {False: 'yes', True: 'no'}}, 'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}


In [15]:
training_data = df_tennis.iloc[1:-4] # all but last thousand instances
test_data = df_tennis.iloc[-4:] # just the last thousand
train_tree = id3(training_data, 'PlayTennis', attribute_names)
print("\n\nThe Resultant Decision train_tree is :\n")
print(train_tree)
test_data['predicted2'] = test_data.apply(classify,axis=1,args=(train_tree,'Yes') )
print ('\n\n Training the model for a few samples, and again predicting \'Playtennis\' for remaining attribute')
print('The Accuracy for new trained data is : ' + str( sum(test_data['PlayTennis']==test_data['predicted2'] ) / (1.0*len(test_data.index)) ))



The Resultant Decision train_tree is :

{'outlook': {'overcast': 'yes', 'rainy': {'windy': {False: 'yes', True: 'no'}}, 'sunny': {'temp': {'cool': 'yes', 'hot': 'no', 'mild': 'no'}}}}


 Training the model for a few samples, and again predicting 'Playtennis' for remaining attribute
The Accuracy for new trained data is : 0.75


C:\Users\ADMIN\AppData\Local\Temp/ipykernel_3396/2245435879.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted2'] = test_data.apply(classify,axis=1,args=(train_tree,'Yes') )
